In [ ]:
# Multimodal Car Bumper Recommendation System

# Import necessary libraries
from vertexai.generative_models import GenerativeModel, Image
import IPython
import typing
import urllib.request
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps

# Initialize the Vertex AI SDK for your project
# Replace '[your-project-id]' with your actual Google Cloud project ID
PROJECT_ID = '[your-project-id]'
LOCATION = 'us-central1'  # You can choose the location nearest to you

import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Load the Gemini 1.0 Pro Vision model
multimodal_model = GenerativeModel('gemini-1.0-pro-vision')

# Helper functions for image handling
def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        if response.headers['Content-Type'] not in ('image/png', 'image/jpeg'):
            raise Exception('Image can only be in PNG or JPEG format')
        return response.read()

def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)

def display_image(image: Image, max_width: int = 600, max_height: int = 350) -> None:
    pil_image = typing.cast(PIL_Image.Image, image._pil_image).convert('RGB')
    image_width, image_height = pil_image.size
    if max_width < image_width or max_height < image_height:
        pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
    ipython_image = IPython.display.Image(data=pil_image.tobytes(), format=pil_image.format)
    IPython.display.display(ipython_image)

# Replace 'your_car_image_url_here' with the URL of your car image
car_image_url = 'your_car_image_url_here'

# Replace these URLs with the actual URLs of your backside bumper options
bumper_image_urls = [
    'bumper_option_1_url_here',
    'bumper_option_2_url_here',
    'bumper_option_3_url_here',
    'bumper_option_4_url_here',
]

# Load images
car_image = load_image_from_url(car_image_url)
bumper_images = [load_image_from_url(url) for url in bumper_image_urls]

# Create content for the multimodal prompt
contents = [
    'Given the following car:',
    car_image,
    'And these backside bumper options:',
    'Option 1:',
    bumper_images[0],
    'Option 2:',
    bumper_images[1],
    'Option 3:',
    bumper_images[2],
    'Option 4:',
    bumper_images[3],
    'Recommend the most suitable backside bumper for the car, considering style, color, and overall aesthetics:',
]

# Generate recommendation
responses = multimodal_model.generate_content(contents, stream=True)

# Display prompt and response
print('-------Prompt--------')
for content in contents:
    if isinstance(content, Image):
        display_image(content)
    else:
        print(content)

print('\n-------Response--------')
for response in responses:
    print(response.text, end='')
